In [30]:
from utils.consts import * ## const variables are there, so paths and extraction features ##
from utils. plotting import * ## plotting ##
from data_reader_babalit import * ## the class which reads balabit dataset ##
from data_reader_chaoshen import * ## the class which reads chaoshen datasets TODO There are several issues with this dataset -> explained later ##
from legality_analyser import * ## the class which analyses the datasets with 0 1 labels --> splitting samples is being done there ##  
import random
from nonlegality_analyser import * ## the class which analyses the datasets without 0 1 labels --> splitting samples is being done there ##  


from custom_dataset import *
from VAEmodel import *


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef

from torch.utils.data import  DataLoader
from torchvision import transforms
from neural_data_creator import *




## Creating dataset

In [31]:

## Specyfing the USERS ## 
users = [7,9,12,15,16,20,21,23,29, 35]
## Creating the object which analyses the dataset ## 
balabit_reader = DataReaderBalabit(BALABIT, users, False) ## the arguments are DATASET, users, supervised, how many records should be procesed ##
balabit_reader.processDataWithoutLabels() ## creating the data without labels ##
## getting path from the reader ##
path = balabit_reader.getFileName()

## creating analyser class ## 
balabitAnalyser = nonLegalityAnalyser(path)
## counting how many actions were extracted ## 
print(balabitAnalyser.countActions())



File already exist
type_of_action     1     3     4
userid                          
7               3885  4090   695
9               2473  2580   288
12              8350  4561   874
15              3332  4796   639
16              5231  6681  1450
20              3297  2723   333
21              3383  3161   528
23              2519  3927   307
29              2683  4871   938
35              2927  2401   380


Dataset Variables

In [32]:
accuracy_final = {}
precision_final= {}
recall_final   = {}
f1_final = {}
mcc_final= {}


In [33]:
test_ratio = 0.2
threshold_ratio = 0.1
input_size = 39  
hidden_size = 128
latent_size = 10
learning_rate = 1e-5
fpr = {}
tpr = {}
roc_auc = {}
precision = {}
recall = {}
average_precision = {}

batch_size = 16 

for trueUser in users:
    dataset = balabitAnalyser.createTrainingDataWithLabel(trueUser)
    neuralDataCreator = NeuralDataCreator(dataset)
    x_training, y_training, x_validation, y_validation, x_threshold_positive, x_threshold_negative, y_threshold, x_test, y_test = neuralDataCreator.create_data(test_ratio, threshold_ratio)


    training_dataset, validation_dataset, treshold_dataset_positive, treshold_dataset_negative , test_dataset = neuralDataCreator.create_datasets(x_training, x_validation , x_threshold_positive, x_threshold_negative, x_test)

## CREATING DATALOADERS ##

    training_loader = DataLoader(training_dataset, batch_size=16, shuffle=True)
    validation_loader = DataLoader(validation_dataset, batch_size=16, shuffle=False)
    threshold_loader_positive = DataLoader(treshold_dataset_positive, batch_size=batch_size, shuffle=False)
    threshold_loader_negative = DataLoader(treshold_dataset_negative, batch_size=batch_size, shuffle=False)

    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = VAE(input_size, hidden_size, latent_size, learning_rate)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    weight = 0.5

    # Optimizer
    num_epochs = 100
    for epoch in range(num_epochs):
        for batch in training_loader:
            model.train()  # Set the model to training mode
            kl_weight = min(epoch / (num_epochs/3), 1.0)
            # kl_weight = 1

            x = batch.to(device)
            x = x.view(-1, input_size)

            # Forward pass
            loss = model.train_step(x, weight, kl_weight)

    
        # Validation phase
        model.eval()
        with torch.no_grad():
            total_validation_loss = 0.0
            num_validation_batches = 0

            for batch in validation_loader:
                x_val = batch.to(device)
                x_val = x_val.view(-1, input_size)
                reconstruction_val, mu_val, log_var_val = model(x_val)
                validation_loss = model.compute_loss(reconstruction_val, x_val, mu_val, log_var_val, weight, kl_weight)

                total_validation_loss += validation_loss.item()
                num_validation_batches += 1

        average_validation_loss = total_validation_loss / num_validation_batches

        # Print the loss at the end of each epoch
        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss:.4f}, Validation Loss: {average_validation_loss:.4f}')


    model.eval()
    artificial_labels  = []
    validation_losses = []
    reconstruction_losses_positive = []
    reconstruction_losses_negative = []
    with torch.no_grad():
        total_validation_loss = 0.0
        num_validation_batches = 0

        # for batch in threshold_loader_positive:
        #     x_val = batch.to(device)
        #     x_val = x_val.view(-1, input_size)
        #     reconstruction_val, mu_val, log_var_val = model(x_val)
        #     validation_loss = model.compute_loss(reconstruction_val, x_val, mu_val, log_var_val, weight, kl_weight)
        #     validation_losses.append(validation_loss.item())
        #     total_validation_loss += validation_loss.item()
        #     num_validation_batches += 1
        #     artificial_labels.append(0)

        # average_validation_loss = total_validation_loss / num_validation_batches

    
        for batch in threshold_loader_negative:
                x_val = batch.to(device)
                x_val = x_val.view(-1, input_size)
                reconstruction_val, mu_val, log_var_val = model(x_val)
                validation_loss = model.compute_loss(reconstruction_val, x_val, mu_val, log_var_val, weight, kl_weight)
                validation_losses.append(validation_loss.item())
                total_validation_loss += validation_loss.item()
                num_validation_batches += 1
                artificial_labels.append(1)
                reconstruction_losses_negative.append(validation_loss.item())

        for batch in threshold_loader_positive:
            x_val = batch.to(device)
            x_val = x_val.view(-1, input_size)
            reconstruction_val, mu_val, log_var_val = model(x_val)
            validation_loss = model.compute_loss(reconstruction_val, x_val, mu_val, log_var_val, weight, kl_weight)
            validation_losses.append(validation_loss.item())
            total_validation_loss += validation_loss.item()
            num_validation_batches += 1
            artificial_labels.append(0)
            reconstruction_losses_positive.append(validation_loss.item())


        average_validation_loss = total_validation_loss / num_validation_batches

        num_samples = num_validation_batches



    thresholds = np.linspace(min(validation_losses), max(validation_losses), 100)
    f1_scores = [f1_score(artificial_labels, validation_losses > t) for t in thresholds]
    # f1_scores = [f1_score(y_artificial_lab, validation_losses > t) for t in thresholds]

    optimal_threshold = thresholds[np.argmax(f1_scores)]



    print(f'F1-Score: {max(f1_scores):.4f}, Optimal Threshold: {optimal_threshold:.4f}')
    fpr[trueUser], tpr[trueUser], threshold = roc_curve(artificial_labels, validation_losses)
    roc_auc[trueUser] = auc(fpr[trueUser], tpr[trueUser])



    precision[trueUser], recall[trueUser], _ = precision_recall_curve(artificial_labels, validation_losses)

# Calculate the area under the precision-recall curve (AUC-PR)
    average_precision[trueUser] = auc(recall[trueUser], precision[trueUser])
    ### OPTIMAL THRESHOLD ROC AUC ##
    optimal_threshold_index_auc = np.argmax(tpr[trueUser] - fpr[trueUser])
    optimal_threshold_ROC = threshold[optimal_threshold_index_auc]




    model.eval()
    validation_losses = []
    iterator = 0
    y_label = []
    with torch.no_grad():
        for batch in test_loader:
            x_val = batch.to(device)
            x_val = x_val.view(-1, input_size)
            reconstruction_val, mu_val, log_var_val = model(x_val)
            validation_loss = model.compute_loss(reconstruction_val, x_val, mu_val, log_var_val, weight, kl_weight)
            validation_losses.append(validation_loss.item())
            y_label.append(y_test[iterator*batch_size])
            iterator+=1




    predictions = (validation_losses < optimal_threshold_ROC).astype(int)

    y_pred = (validation_losses < optimal_threshold_ROC).astype(int)

    # Confusion matrix
    conf_matrix = confusion_matrix(y_label, y_pred)

    # Accuracy
    accuracy_final[trueUser] = accuracy_score(y_label, y_pred)

    # Precision
    precision_final[trueUser] = precision_score(y_label, y_pred)

    # Recall
    recall_final[trueUser] = recall_score(y_label, y_pred)

    # F1 Score
    f1_final[trueUser] = f1_score(y_label, y_pred)

    # Matthews correlation coefficient
    mcc_final[trueUser] = matthews_corrcoef(y_label, y_pred)

 


Epoch [1/100], Training Loss: 52.7410, Validation Loss: 113.2240
Epoch [2/100], Training Loss: 34.7148, Validation Loss: 81.2177
Epoch [3/100], Training Loss: 24.8681, Validation Loss: 58.0035
Epoch [4/100], Training Loss: 18.9643, Validation Loss: 43.1732
Epoch [5/100], Training Loss: 11.5521, Validation Loss: 36.6422
Epoch [6/100], Training Loss: 16.4522, Validation Loss: 33.9157
Epoch [7/100], Training Loss: 11.6571, Validation Loss: 32.2326
Epoch [8/100], Training Loss: 12.0599, Validation Loss: 30.9948
Epoch [9/100], Training Loss: 12.2028, Validation Loss: 29.8918
Epoch [10/100], Training Loss: 11.2136, Validation Loss: 28.6544
Epoch [11/100], Training Loss: 11.0995, Validation Loss: 27.9495
Epoch [12/100], Training Loss: 9.9367, Validation Loss: 27.0511
Epoch [13/100], Training Loss: 7.6093, Validation Loss: 26.7078
Epoch [14/100], Training Loss: 12.3271, Validation Loss: 26.3467
Epoch [15/100], Training Loss: 10.5608, Validation Loss: 25.9805
Epoch [16/100], Training Loss: 7.13

## TESTING ON DATA ##

In [34]:
roc_path = f'wykresy\\VAEBalabit_balanced{batch_size}ROC'
pr_path = f'wykresy\\VAEBalabit_balanced{batch_size}PR'
plotROCs(fpr, tpr, roc_auc, users,False, True, roc_path)
plot_precisions_recalls(precision, recall, average_precision, True, pr_path)

In [35]:
accuracy_final



{7: 0.963302752293578,
 9: 0.9402985074626866,
 12: 0.838150289017341,
 15: 0.7909090909090909,
 16: 0.8562874251497006,
 20: 0.7375,
 21: 0.9101123595505618,
 23: 0.6823529411764706,
 29: 0.719626168224299,
 35: 0.9583333333333334}

In [36]:
precision_final


{7: 0.9473684210526315,
 9: 0.8947368421052632,
 12: 0.8734177215189873,
 15: 0.82,
 16: 0.8658536585365854,
 20: 0.9523809523809523,
 21: 0.9512195121951219,
 23: 0.75,
 29: 0.7222222222222222,
 35: 0.9230769230769231}

In [37]:
recall_final


{7: 0.9818181818181818,
 9: 1.0,
 12: 0.7931034482758621,
 15: 0.7454545454545455,
 16: 0.8452380952380952,
 20: 0.5,
 21: 0.8666666666666667,
 23: 0.5581395348837209,
 29: 0.7222222222222222,
 35: 1.0}

In [38]:
f1_final


{7: 0.9642857142857142,
 9: 0.9444444444444444,
 12: 0.8313253012048193,
 15: 0.780952380952381,
 16: 0.8554216867469879,
 20: 0.6557377049180327,
 21: 0.9069767441860465,
 23: 0.6399999999999999,
 29: 0.7222222222222222,
 35: 0.9600000000000001}

In [39]:
mcc_final

{7: 0.9271996650720626,
 9: 0.8867265032393299,
 12: 0.6793715760320801,
 15: 0.5842373946721772,
 16: 0.7128102210768651,
 20: 0.5397814402407793,
 21: 0.8237120116864876,
 23: 0.3793990878450622,
 29: 0.4392033542976939,
 35: 0.9198662110077999}

In [40]:
roc_auc

{7: 0.9821487603305785,
 9: 0.9316608996539791,
 12: 0.9553441669969612,
 15: 0.9047933884297521,
 16: 0.9384920634920635,
 20: 0.9975,
 21: 0.9520987654320987,
 23: 0.6976744186046512,
 29: 0.8415637860082305,
 35: 0.9544753086419753}